*Kaito Arnoni Lacerda* - *Parcours Ingénieur IA*

# <font color=black size=6> P10 - DÉVELOPPEZ UN CHATBOT POUR RÉSERVER DES VACANCES </font> #


# <font color=black size=5> I - Préparation d’exemples LUIS </font> #

###  <font size=4 color="black"> SOMMAIRE </font>

<a href="#1" style="text-decoration:none"> <font size=2> 1. INTRODUCTION</a></font>

<a href="#2" style="text-decoration:none"> <font size=2> 2. MÉTHODOLOGIE</a></font>

<a href="#3" style="text-decoration:none"> <font size=2> 3. LE JEU DE DONNÉES</a></font>

###  <font size=4 color="#3498DB "> 1. INTRODUCTION <a name="1"></a> </font>

Fly Me est une agence qui propose des voyages clé en main pour les particuliers ou les professionnels. 
Dans ce contexte cette société a lancé un projet ambitieux de développement d’un chatbot pour aider les utilisateurs à choisir une offre de voyage.

**MISSION**

La première étape de ce projet est de construire un MVP qui aidera les employés de Fly Me à réserver facilement un billet d’avion pour leurs vacances.

Ce premier MVP va permettre de tester rapidement et à grande échelle le concept et les performances du chatbot. Il devra également être capable d'identifier dans la demande de l’utilisateur les cinq éléments suivants :

* Ville de départ
* Ville de destination
* Date aller souhaitée du vol
* Date retour souhaitée du vol
* Budget maximum pour le prix total des billets.

Si un des éléments est manquant, le chatbot devra pouvoir poser les questions pertinentes (en anglais) à l’utilisateur pour comprendre complètement sa demande. 

###  <font size=4 color="#3498DB  "> 2. MÉTHODOLOGIE <a name="2"></a> </font>

**Organisation du projet**

Ce projet a été conçu sur la méthodologie MLOps et sera organisé en 3 parties:

- Préparation des données: préparation d’exemples LUIS à partir du jeu de données disponible pour entraînement du modèle.
- Création et entraînement du modèle LUIS.
- Développement du chatbot avec le Bot Framework.
- Gestion de la performance avec Insights.
- Tests unitaires et mise en production du chatbot.

**Architecture MLOps**

Le MLOps est une culture et une pratique d'ingénierie ML qui vise à unifier le développement de systèmes ML et leur mise en opération (Ops). Appliquer le MLOps signifie que l'on vise l'automatisation et la surveillance de toutes les étapes de la construction d'un système de ML, y compris l'intégration, les tests, la publication, le déploiement et la gestion de l'infrastructure.

Nous avons donc mis en place une architecture MLOps afin de pouvoir itérer rapidement sur les versions de notre MVP.

<img src="MLOps.png" alt="Architecture MLOps" width="900"/>
<p style="text-align: center;">Architecture MLOps.</p>

###  <font size=4 color="#3498DB  "> 3. LE JEU DE DONNÉES <a name="3"></a> </font>

Nous allons utiliser un jeu de données fournit gratuitement par Microsoft.
Il est composé d'un fichier JSON contenant 1369 dialogues entre 2 humains :
* Le 1er joue le rôle de l’utilisateur et le 2ème se fait passer pour le bot.
* L’utilisateur veut booker un vol et/ou un hôtel.
* Le bot va récupérer les paramètres de recherche et proposer une offre.
* L’utilisateur va donner une note représentant sa satisfaction.
* Effectué par 12 participants sur 20 jours.

Les entités dans les textes des utilisateurs sont déjà labellisées :

* Ville de départ.
* Ville d’arrivée.
* Heure de départ.
* Heure d’arrivée.
* Budget.
* Etc…

###  <font size=3 color="#3498DB "> 3.1 TÉLÉCHARGEMENT DE DONNÉES </font>
Nous commençons par télécharger et extraire les données.

In [54]:
from notebook_p10 import *

In [6]:
# Lien vers le dataset
DATASET_URL = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+10%C2%A0-+D%C3%A9veloppez+un+chatbot+pour+r%C3%A9server+des+vacances/frames.zip"
JSON_PATH = "/Users/kaju/Documents/FORMACAO IA OPENCLASSROOMS/PROJET 10 - DEVELOPPEZ UN CHATBOT POUR RESERVER DES VACANCES/P10_ARNONI_KAITO/data/json"
file_data = "frames.json"

# On vérifie si le fichier est bien présent
if file_data in os.listdir(JSON_PATH):
    print("Tous les fichiers sont bien présents.")
# Sinon on télécharge et on extrait le fichier
else:
    print("Téléchargement des données en cours...")

    # On télécharge le .zip dans un fichier temporaire et on extrait les données
    tmp, _ = urllib.request.urlretrieve(DATASET_URL)
    with zipfile.ZipFile(tmp, "r") as f:
        # On extrait le fichier json
        f.extract(file_data, JSON_PATH)

    # On supprime le fichier temporaire
    urllib.request.urlcleanup()
    
    print("Téléchargement des données terminé.")

Téléchargement des données en cours...
Téléchargement des données terminé.


###  <font size=3 color="#3498DB  "> 3.2 EXPLORATION ET ANALYSE DE DONNÉES </font>

Nous allons commencer par charger les jeux de données frame.json. Cet jeu de données n'est constitué que d'un seul fichier JSON. Il s'agit d'une liste de dialogues appelées frames.

In [11]:
# On charge les frames
with open(JSON_PATH + "/frames.json") as f:
    frames = json.load(f)

In [12]:
len(frames)

1369

In [13]:
frames[0]

{'user_id': 'U22HTHYNP',
 'turns': [{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
   'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'active_frame': 1,
    'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
 

On constate que le jeu de données a 1369 frame et chaque frame contient un `user_id` et une liste labellisée `turns`.

###  <font size=3 color="#3498DB  "> 3.3 ANALYSE DE DONNÉES </font>

Tout d'abord nous allons regarder la conversation entre l'utilisateur et le bot dans le frame [0].

In [29]:
for n, i in enumerate(frames[0]["turns"]):
    print(str(n+1)+'.', i['author'].upper(), ':', i['text'])
    if n%2 ==0:
        print('- -'*38)
    else:
        print('*'*115)

1. USER : I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
2. WIZARD : Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option?
*******************************************************************************************************************
3. USER : Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
4. WIZARD : I checked the availability for this date and there were no trips available.  Would you like to select some alternate dates?
**************************************************************************

On vérifie que le premier texte contient les informations plus relevantes (la date, le départ et la destination et le budget) pour l'entraînement de notre modèle et les autres textes correspondant aux échanges complémentaires avec le Bot. Donc nous n'allons prendre en compte que le premier texte de chaque frame.

**Analyse des labels**

Il est aussi intéressant de connaitre les labels existant dans chaque turn. Pour le premier nous avons:

In [46]:
labels = frames[0]["turns"][0]["labels"]["acts_without_refs"]
print(json.dumps(labels, indent=2))

[
  {
    "args": [
      {
        "val": "book",
        "key": "intent"
      }
    ],
    "name": "inform"
  },
  {
    "args": [
      {
        "val": "Atlantis",
        "key": "dst_city"
      },
      {
        "val": "Caprica",
        "key": "or_city"
      },
      {
        "val": "Saturday, August 13, 2016",
        "key": "str_date"
      },
      {
        "val": "8",
        "key": "n_adults"
      },
      {
        "val": "1700",
        "key": "budget"
      }
    ],
    "name": "inform"
  }
]


* `1er liste`: l'intent indiquent l'intention principale du texte.

* `2ème liste`: les informations détaillées de l'intent.

Intéressons nous maintenant en decouvir toutes les label dans le jeu de donnes frame.

In [42]:
label_values = defaultdict(list)
for frame in frames:
    for turn in frame["turns"]:
        # On ne s'intéresse qu'aux textes de l'utilisateur
        if turn["author"] != "user":
            continue

        for args in turn["labels"]["acts_without_refs"]:
            # On va ajouter chaque valeur dans la liste du label correspondant
            for label in args["args"]:
                try:
                    label_values[label["key"]].append(label.get("val", ""))
                except:
                    print(label)
                    
# On crée un dataframe
labels = pd.Series(label_values).to_frame("all_values")

# On ajoute des variables concernant toutes les valeurs de chaque label
labels["all_values_nb"] = labels["all_values"].apply(len)
all_values_total = labels["all_values_nb"].sum()
labels["all_values_ratio"] = labels["all_values_nb"] / all_values_total

# On ajoute des variables concernant les valeurs uniques de chaque label
labels["unique_values"] = labels["all_values"].apply(set)
labels["unique_values_nb"] = labels["unique_values"].apply(len)

# On range les données par plus effectif
labels = labels.sort_values("all_values_nb", ascending=False)

labels.shape

(50, 5)

In [43]:
labels

,all_values,all_values_nb,all_values_ratio,unique_values,unique_values_nb
dst_city,"[Atlantis, Neverland, Mos Eisley, Neverland, M...",3541,0.202795,"{SACRAMENTO, Santios, Canada, Spain, monterrey...",364
intent,"[book, book, book, book, book, book, book, boo...",2006,0.114885,{book},1
or_city,"[Caprica, Caprica, Atlantis, Caprica, Gotham C...",1962,0.112365,"{monterrey, santos, portland, Jerusalem, Burli...",333
str_date,"[Saturday, August 13, 2016, August 13, 2016, -...",1544,0.088426,"{August 23, September 14th, on or after septem...",447
budget,"[1700, 1900, 2100, None, $2500, $2100, $2200, ...",1246,0.071359,"{$1000, $5100, 3700, 29000, less than 3100, ch...",395
n_adults,"[8, 5, 5, 2, 2, 3, with 14 adults, 1, 12, Just...",1029,0.058931,"{all, 8, me and Madonna, 25, my wife, me, her ...",262
end_date,"[Wednesday August 31, 2016, Sunday August 28th...",1022,0.058530,"{September 14th, 25, September 15th, August 22...",320
ref_anaphora,"[that's, this package, that, options, the hote...",874,0.050054,"{other package, this is, first one, this, seco...",149
category,"[None, 4-star, None, 3, None, None, None, None...",690,0.039517,"{as nice as possible, lavishly, lower end, Hig...",85
price,"[None, None, None, None, None, None, too expen...",539,0.030869,"{2890, $577.72USD, under 3600, $110.93, cheape...",49


Rappelons-nous que notre modèle LUIS devra être capable d'identifier dans la demande de l’utilisateur les cinq éléments suivants :
* Ville de départ
* Ville de destination
* Date aller souhaitée du vol
* Date retour souhaitée du vol
* Budget maximum pour le prix total des billets.

Donc les 5 labels qui vont nous intéresser sont: 

* `or_city` : ville de départ
* `dst_city` : ville de destination
* `str_date` : date aller souhaitée du vol
* `end_date` : date retour souhaitée du vol
* `budget` : budget maximum pour le prix total des billets.

Toutes ces 5 labels qui nous intéressent sont parmis les 7 labels les plus représentés.

###  <font size=3 color="#3498DB  "> 3.4 TRANSFORMATION DES DONNÉES EN FORMAT LUIS </font>

Dans cette partie nous avons l'objectif d'obtenir un jeu d'entrainement et en un jeu de test dans le format LUIS. 

Regardons un exemple du format LUIS:

In [ ]:
# Define labeled example
labeledExampleUtteranceWithMLEntity = {
    "text": "I want two small seafood pizzas with extra cheese.",
    "intentName": intentName,
    "entityLabels": [
        {
            "startCharIndex": 7,
            "endCharIndex": 48,
            "entityName": "Pizza order",
            "children": [
                {
                    "startCharIndex": 7,
                    "endCharIndex": 30,
                    "entityName": "Pizza",
                    "children": [
                        {
                            "startCharIndex": 7,
                            "endCharIndex": 9,
                            "entityName": "Quantity"
                        },
                        {
                            "startCharIndex": 11,
                            "endCharIndex": 15,
                            "entityName": "Size"
                        },
                        {
                            "startCharIndex": 17,
                            "endCharIndex": 23,
                            "entityName": "Type"
                        }]
                },
                {
                    "startCharIndex": 37,
                    "endCharIndex": 48,
                    "entityName": "Toppings",
                    "children": [
                        {
                            "startCharIndex": 37,
                            "endCharIndex": 41,
                            "entityName": "Quantity"
                        },
                        {
                            "startCharIndex": 43,
                            "endCharIndex": 48,
                            "entityName": "Type"
                        }]
                }
            ]
        }
    ]
}

Pour y arriver nous allons suivre les étapes suivantes:

- Filtrer les données qui nous intéressent.
- Transformer les données en format LUIS.
- Séparer les données en jeu d'entrainement et jeu de test.
- Enregistrer les données dans le Datastore.

Dans notre modèle LUIS déjà crée nous avons les entities suivantes: `budget`, `from_city`, `from_dt`, `to_city`, `to_dt`. Donc nous allons attribuer un des 5 labels choisis precedement à chaque entité de notre modèle.


In [55]:
# Mapping entre les labels du jeu de données et ceux de LUIS
label_to_entity = {
    "or_city": "from_city",
    "dst_city": "to_city",
    "str_date": "from_dt",
    "end_date": "to_dt",
    "budget": "budget"
}


# On convertit les turns utilisateur du jeu de données pour LUIS
df = turns_to_LUIS(
    frames,
    intent_name = "book_flight",
    label_to_entity = label_to_entity
)
    
df.shape

(10407, 11)

In [56]:
df.head()

,user_turn_id,text,intent,entities,entity_total_nb,from_city_nb,to_city_nb,from_dt_nb,to_dt_nb,budget_nb,text_word_nb
0,0,I'd like to book a trip to Atlantis from Capri...,book_flight,"[{'entity': 'to_city', 'startPos': 27, 'endPos...",4,1,1,1,0,1,25
1,1,"Yes, how about going to Neverland from Caprica...",None,"[{'entity': 'to_city', 'startPos': 24, 'endPos...",4,1,1,1,0,1,23
2,2,I have no flexibility for dates... but I can l...,None,"[{'entity': 'from_city', 'startPos': 56, 'endP...",2,2,0,0,0,0,18
3,3,I suppose I'll speak with my husband to see if...,None,[],0,0,0,0,0,0,25
4,0,"Hello, I am looking to book a vacation from Go...",book_flight,"[{'entity': 'to_city', 'startPos': 59, 'endPos...",3,1,1,0,0,1,16


Nous constatons que les textes 1 et 2 sont labellisés avec intent none malgré le fait de contenir des labels qui nous intéressent. Pour régler ce problème nous allons attribuer l'intention book flight à tous les textes ayant au moins un label d'intérêt.

In [59]:
def change_intent(df):
    if df["intent"]=="None" and df["entity_total_nb"]==0:
        return 'None'
    else:
        return 'book_flight'

df["intent"] = df.apply(change_intent, axis=1)
df.head()

,user_turn_id,text,intent,entities,entity_total_nb,from_city_nb,to_city_nb,from_dt_nb,to_dt_nb,budget_nb,text_word_nb
0,0,I'd like to book a trip to Atlantis from Capri...,book_flight,"[{'entity': 'to_city', 'startPos': 27, 'endPos...",4,1,1,1,0,1,25
4,0,"Hello, I am looking to book a vacation from Go...",book_flight,"[{'entity': 'to_city', 'startPos': 59, 'endPos...",3,1,1,0,0,1,16
14,0,Hello there i am looking to go on a vacation w...,book_flight,"[{'entity': 'to_city', 'startPos': 63, 'endPos...",1,0,1,0,0,0,20
25,0,"Hi I'd like to go to Caprica from Busan, betwe...",book_flight,"[{'entity': 'to_city', 'startPos': 21, 'endPos...",4,1,1,1,1,0,19
32,0,"Hello, I am looking to book a trip for 2 adult...",book_flight,"[{'entity': 'budget', 'startPos': 67, 'endPos'...",3,1,1,0,0,1,25


In [60]:
df["intent"].value_counts()

book_flight    1260
None             69
Name: intent, dtype: int64

In [62]:
for text in df[df["intent"] == "None"]["text"].iloc[:10]:
    print(text)

Hi!
Heyo!
Good morning.
Hello wozbot!
ay whats up?
hi there
me again... I'm still burnt out from work
hey
hello hello
HEY


Comme nous avons déjà discuté précédemment, nous n'allons prendre en compte que le premier texte de chaque turn. Nous allons supprimer aussi les doublons.

In [61]:
# On s'interesse 
df = df[df["user_turn_id"] == 0]

# On supprime les doublons
df = df.drop_duplicates(["text"])

**Indicateurs statistiques**

In [63]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_turn_id,1329.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
entity_total_nb,1329.0,1.860798,1.272167,0.0,1.0,2.0,3.0,5.0
from_city_nb,1329.0,0.607223,0.490091,0.0,0.0,1.0,1.0,2.0
to_city_nb,1329.0,0.651618,0.476637,0.0,0.0,1.0,1.0,1.0
from_dt_nb,1329.0,0.278405,0.448382,0.0,0.0,0.0,1.0,1.0
to_dt_nb,1329.0,0.159518,0.366297,0.0,0.0,0.0,0.0,1.0
budget_nb,1329.0,0.164033,0.370445,0.0,0.0,0.0,0.0,1.0
text_word_nb,1329.0,17.469526,11.829137,1.0,9.0,15.0,23.0,74.0


Nous pouvons conclure que:

- la ville de départ et la ville d'arrivée sont suivantes indiquées dans le premier texte de l'utilisateur, avec un taux de présence de 61% et 65% de cas respectivement.
- par contre la date de départ, la date d'arrivée et le budget sont moins fréquents, présents en 28%, 16% et 16% de cas respectivement.

Notre MVP devra donc être capable de poser des questions complémentaires aux utilisateurs pour avoir tous les donnés nécessaires.

Pour entrainer et tester notre modèle nous allons prendre tous les textes avec tous les 5 lebels, plus quelques textes avec intention book flight pris au hasard jusqu'à avoir 200 textes. Nous allons prendre aussi tous les textes ayant l'intention `none`.

In [64]:
book_flight_exemples = df[(df["intent"] == "book_flight") & (df["entity_total_nb"] >= 5)]
book_flight_exemples.shape

(33, 11)

In [65]:
# Plus 167 textes au hazard
tmp = df[(df["intent"] == "book_flight") & (df["entity_total_nb"] < 5)]
tmp = tmp.sample(n=167, random_state=25)

# On concatène les données
book_flight_exemples = pd.concat([book_flight_exemples, tmp])
book_flight_exemples.shape

(200, 11)

In [67]:
# On ajoute les textes intent None :
book_flight_exemples = pd.concat([book_flight_exemples,
                     df[df["intent"] == "None"]])
book_flight_exemples.shape

(269, 11)

In [68]:
book_flight_exemples['intent'].value_counts()

book_flight    200
None            69
Name: intent, dtype: int64

###  <font size=3> SÉPARER LES DONNÉES </font>

Nous allons dans cette étape créer deux jeux de données LUIS, un jeu d'entrainement (70% des données) et un jeu de test (30% des données).

In [72]:
exemples_train = book_flight_exemples.sample(frac=0.7, random_state=42)
exemples_test = book_flight_exemples.drop(exemples_train.index)


print(f"Jeu d'entrainement: {exemples_train.shape[0]} textes labellisés.")
print(f"Jeu de test: {exemples_test.shape[0]} textes labellisés.")

Jeu d'entrainement: 188 textes labellisés.
Jeu de test: 81 textes labellisés.


Finalement nous allons les transformer en format LUIS et les sauvegarder.

In [73]:
utterances_train = exemples_train[["text", "intent", "entities"]].to_dict("records")

print(json.dumps(utterances_train[0], indent=2))

{
  "text": "I would like to find a vacation between the 24th and 27th from San Francisco to Fort Lauderdale for 3800",
  "intent": "book_flight",
  "entities": [
    {
      "entity": "from_dt",
      "startPos": 44,
      "endPos": 47,
      "children": []
    },
    {
      "entity": "to_dt",
      "startPos": 53,
      "endPos": 56,
      "children": []
    },
    {
      "entity": "from_city",
      "startPos": 63,
      "endPos": 75,
      "children": []
    },
    {
      "entity": "to_city",
      "startPos": 80,
      "endPos": 94,
      "children": []
    },
    {
      "entity": "budget",
      "startPos": 100,
      "endPos": 103,
      "children": []
    }
  ]
}


In [75]:
utterances_test = exemples_test[["text", "intent", "entities"]].to_dict("records")
utterances_test = {
    "LabeledTestSetUtterances": utterances_test
}

print(json.dumps(utterances_test["LabeledTestSetUtterances"][0], indent=2))

{
  "text": "Hey there, I\u2019m looking to check out a few destinations to see if they\u2019ll be appropriate for me to accept an internship in. I want to travel from August 27-30, just enough to get a taste of each location.\nI am leaving from Monterrey and I would like to spend 5200 at most, as my internship is unpaid.\nIs there anything available to Kobe for these dates and price?",
  "intent": "book_flight",
  "entities": [
    {
      "entity": "to_city",
      "startPos": 331,
      "endPos": 334,
      "children": []
    },
    {
      "entity": "from_city",
      "startPos": 222,
      "endPos": 230,
      "children": []
    },
    {
      "entity": "budget",
      "startPos": 258,
      "endPos": 261,
      "children": []
    },
    {
      "entity": "from_dt",
      "startPos": 145,
      "endPos": 153,
      "children": []
    },
    {
      "entity": "to_dt",
      "startPos": 155,
      "endPos": 156,
      "children": []
    }
  ]
}


###  <font size=3> ENREGISTRER LES DONNÉES</font>

In [77]:
JSON_PATH_LUIS = "/Users/kaju/Documents/FORMACAO IA OPENCLASSROOMS/PROJET 10 - DEVELOPPEZ UN CHATBOT POUR RESERVER DES VACANCES/P10_ARNONI_KAITO/ARNONI_KAITO_2_LUIS_042022/"
    
# On enregistre les données
file_path = os.path.join(JSON_PATH_LUIS, "utterances_train.json")
with open(file_path, "w") as f:
    json.dump(utterances_train, f)
        
file_path = os.path.join(JSON_PATH_LUIS, "utterances_test.json")
with open(file_path, "w") as f:
    json.dump(utterances_test, f)